# Matrix Multiplication From Scratch
---

## Import Libraries

In [40]:
import torch
import timeit
import operator
from functools import partial

## Test Framework

In [93]:
def test(a, b, compare, compare_name=None):
    if compare_name is None:
        compare_name = compare.__name__
    assert compare(a, b),\
    f"{compare_name} check failed:\n{a}\n{b}"

def test_equality(a, b):
    test(a, b, operator.eq, "Equality")

def test_approximately(a, b):
    allclose = partial(torch.allclose, atol=1e-5, rtol=1e-03)
    if not isinstance(a, torch.Tensor) or not isinstance(b, torch.Tensor):
        a = torch.tensor(a)
        b = torch.tensor(b)
    test(a, b, allclose, "Approximate Equality")

In [94]:
test_equality(1e-5,1e-5)

In [95]:
test_approximately(1e-5, 1e-6)

# Optimizations in Matrix Multiplication

**Toy Variables**

In [96]:
a = torch.tensor([[1.,2.,1.],
                  [2.,3.,2.],
                  [3.,1.,3.]])

b = torch.tensor([[1., 2.],
                  [2., 1.],
                  [1., 2.]])

In [97]:
a@b

tensor([[ 6.,  6.],
        [10., 11.],
        [ 8., 13.]])

**Test Variables**

In [98]:
A = torch.randn([64,32])
B = torch.randn([32,64])

In [99]:
(A@B).shape

torch.Size([64, 64])

## For Loop

In [112]:
def matmul(a,b):
    ar, ac = a.shape
    br, bc = b.shape
    assert ac==br,\
    f"Inner dimensions must match: {ac} not equal to {br}"
    c = torch.zeros([ar, bc])
    for i in range(ar):
        for j in range(bc):
            for k in range(ac):
                c[i,j] += a[i,k]*b[k,j]
    return c

In [113]:
matmul(a,b)

tensor([[ 6.,  6.],
        [10., 11.],
        [ 8., 13.]])

In [114]:
test_approximately(matmul(A, B), (A@B))

In [115]:
matmul_loop_time = timeit.timeit(partial(matmul,A,B), number=10)

## Array Slicing

In [116]:
def matmul(a,b):
    ar, ac = a.shape
    br, bc = b.shape
    assert ac==br,\
    f"Inner dimensions must match: {ac} not equal to {br}"
    c = torch.zeros([ar, bc])
    for i in range(ar):
        for j in range(bc):
            c[i,j] += (a[i,:]*b[:,j]).sum()
    return c

In [117]:
matmul(a,b)

tensor([[ 6.,  6.],
        [10., 11.],
        [ 8., 13.]])

In [118]:
test_approximately(matmul(A,B), (A@B))

In [119]:
matmul_slice_time = timeit.timeit(partial(matmul,A,B), number=10)

## Array Broadcasting

In [123]:
def matmul(a,b):
    ar, ac = a.shape
    br, bc = b.shape
    assert ac==br,\
    f"Inner dimensions must match: {ac} not equal to {br}"
    c = torch.zeros([ar, bc])
    for i in range(ar):
        c[i,:] += (a[i,:]*b.t()).sum(1)
    return c

In [124]:
matmul(a,b)

tensor([[ 6.,  6.],
        [10., 11.],
        [ 8., 13.]])

In [125]:
test_approximately(matmul(A, B), (A@B))

In [127]:
matmul_broadcast_time = timeit.timeit(partial(matmul,A,B), number=10)

## Einstein Sum

In [129]:
def matmul(a,b):
    ar, ac = a.shape
    br, bc = b.shape
    assert ac==br,\
    f"Inner dimensions must match: {ac} not equal to {br}"
    c = torch.zeros([ar, bc])
    c = torch.einsum("ik,kj->ij", a, b)
    return c

In [130]:
matmul(a,b)

tensor([[ 6.,  6.],
        [10., 11.],
        [ 8., 13.]])

In [131]:
test_approximately(matmul(A,B), (A@B))

In [132]:
matmul_einsum_time = timeit.timeit(partial(matmul,A,B), number=10)

## BLAS in PyTorch

In [135]:
matmul_blas_time = timeit.timeit(partial(torch.matmul,A,B), number=10)

# Performance Comparison

## Improvement with array slicing
How fast is matrix multiplication with array slicing in two nested loops compared to element wise product in three nested loops?

In [138]:
matmul_loop_time, matmul_slice_time

(39.48820200000773, 1.7050149999995483)

**Basic matrix multiplication of a 64x32 with 32x64 matrix in python takes about 40 seconds!**

In [120]:
matmul_loop_time/matmul_slice_time

23.160032023189352

**Array slicing is about 23 times faster than element wise product in three nested loops.**

## Improvement with array broadcasting
How fast is matrix multiplication with broadcasting compared to array slicing in two nested loops?

In [128]:
matmul_slice_time/matmul_broadcast_time

55.417005225958505

**Array broadcasting is about 55 times faster than array slicing in two nested loops.**

## Improvement with einstein sum
How fast is matrix multiplication with einstein sum compared to array broadcasting?

In [139]:
matmul_broadcast_time/matmul_einsum_time

10.303750815109495

**Einstein sum is about 10 times faster than array broadcasting.**

In [134]:
matmul_loop_time/matmul_einsum_time

13224.448061042098

**Matrix multiplication with einstein sum is about 13,000 times faster than multiplication using loops in python!**

## Improvement with PyTorch
How fast is matrix multiplication in PyTorch compared to doing it standard python?

In [136]:
matmul_loop_time/matmul_blas_time

44120.89585891842

**Matrix multiplication in PyTorch is about 44,000 times faster than using standard Python!**

___